In [9]:
import importlib
from collections import defaultdict, deque

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from kaggle_environments import make

%matplotlib inline

In [10]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

In [11]:
e = env_module.Environment()
e.reset()

In [12]:
# while not e.terminal():
for _ in range(50):
    obs = e.observation()
    actions = {}
    for player in e.turns():
        actions[player] = e.rule_based_action_smart_geese(player)
    e.step(actions)
    rewards = e.reward()
# e.outcome()

In [13]:
obs = e.observation()

In [14]:
field = obs[8:13].sum(0)
field

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 1.],
       [0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.]], dtype=float32)

In [15]:
def around(a):
    a_ = [
        ((a[0] - 1) % 7, a[1]),
        ((a[0] + 1) % 7, a[1]),
        (a[0], (a[1] - 1) % 11),
        (a[0], (a[1] + 1) % 11),
    ]
    return [e for e in a_ if field[e[0], e[1]] == 0]

In [16]:
heads = np.where(obs[0] == 1)

for r, c in zip(heads[0], heads[1]):
    break

print(f"{r}, {c}")

5, 10


In [17]:
v = around((r, c))
v

[(4, 10), (6, 10), (5, 0)]

In [18]:
def bfs(v):
    q = deque([(r, c)])
    movable = np.zeros([7, 11])
    searched = defaultdict(bool)
    while len(q) != 0:
        v = q.popleft()
        movable[v] = 1
        searched[v] = True
        edges = [a for a in around(v) if not searched[a]]
        for edge in edges:
            q.append(edge)
    return movable

In [22]:
bfs(v)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1.],
       [1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.],
       [1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1.]])

In [20]:
obs[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)